# Init

In [ ]:
!sudo apt update
!pip install pyspark

In [ ]:
# Добавляем Hive таблицу employee
from pyspark.sql import SparkSession
spark = SparkSession \
       .builder \
       .master('local[*]') \
       .appName("Our First Spark Example") \
       .enableHiveSupport() \
       .getOrCreate()
spark.read.parquet('employee.parquet').repartition(1).write.saveAsTable("employee")
spark.stop()

# Старт сессии и импорт

Импортируем необходимые библиотеки

Документация PySpark: https://spark.apache.org/docs/latest/api/python/index.html

Документация PySpark SQL Functions: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [ ]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [ ]:
# Создание спарк сессии
spark = SparkSession \
       .builder \
       .master('local[*]') \
       .appName("Our First Spark Example") \
       .enableHiveSupport() \
       .getOrCreate()

spark

#Input Data

1) Создать собственный DataFrame

2) Считать из Hive-таблицы

3) Считать из файла любого формата

In [ ]:
# Создаем собственный DataFrame
sample_data = [
    {"name": "John D.",   "age": 30},
    {"name": "Alice G.",  "age": 25},
    {"name": "Bob T.",    "age": 35},
    {"name": "EvecA.",    "age": 28}
    ]

sample_df = spark.createDataFrame(sample_data)
sample_df.show()

Читает DataFrame из Hive-таблицы

In [ ]:
# Смотрим, какие таблицы есть в Hive metastore
spark.sql("show tables").show()

In [ ]:
employee_h_df = spark.read.table("employee")
employee_h_df

In [ ]:
employee_h_df.show(5)

Читает DataFrame из файла parquet

In [ ]:
# Смотрим, какие файлы лежат в нашей директории (linux-командой)
!ls -la | grep employee

In [ ]:
# Читаем parquet-файл
employee_p_df = spark.read.parquet('employee.parquet')
employee_p_df

In [ ]:
employee_p_df.show(5)

Читает DataFrame из файла csv

In [ ]:
# Смотрим, какие файлы лежат в нашей директории (linux-командой)
!ls -la | grep employee

In [ ]:
# Читаем первые 10 строк файла (linux-командой)
!head employee_data.csv

In [ ]:
# Определяем схему для витрины "Сотрудники"
schema = StructType([
    StructField("Id", IntegerType(), False),  # Уникальный идентификатор
    StructField("LastName", StringType(), False),  # Фамилия
    StructField("FirstName", StringType(), False),  # Имя
    StructField("MiddleName", StringType(), True),  # Отчество
    StructField("Gender", StringType(), False),  # Пол
    StructField("Age", IntegerType(), False),  # Возраст
    StructField("Department", StringType(), False),  # Отдел
    StructField("Position", StringType(), False),  # Должность
    StructField("Grade", StringType(), False),  # Грейд
    StructField("HireDate", DateType(), False),  # Дата найма
    StructField("OfferAmount", DoubleType(), False),  # Сумма оффера
    StructField("Education", StringType(), False),  # Образование
    StructField("CurrentStatus", StringType(), False)  # Статус сотрудника
])

# Читаем данные из CSV файла
employee_df = spark.read.csv('employee.csv', schema=schema, header=True)
employee_df

In [ ]:
# Показать схему DataFrame
employee_df.printSchema()

In [ ]:
# Вывести 5 строк (не сокращая)
employee_df.show(5, False)

In [ ]:
# Вывести основные метрики по Int атрибутам
employee_df.select("Id", "Age", "OfferAmount").describe().show()

In [ ]:
# Переводим DF в temp view (для обращения через spark.sql)
employee_df.createOrReplaceTempView("employee_df")

In [ ]:
# Посмотрим, появилась ли таблица в Hive metastore
spark.sql("show tables").show()

In [ ]:
# Выведем select всех записей
spark.sql("select * from employee").show()

In [ ]:
# Выведем ФИО и позицию только активных сотрудников
# employee_df.filter("CurrentStatus = 'Активен'").select("LastName", "FirstName", "MiddleName", "Position").show(10, False)
spark.sql("""
select
  LastName,
  FirstName,
  MiddleName,
  Position
from employee
where CurrentStatus = 'Активен'
          """).show(10, False)

In [ ]:
# Выведем количество уволенных и активных сотрудников
# employee_df.groupBy("CurrentStatus").agg(F.count("LastName").alias("Count")).show()
spark.sql("""
select
  CurrentStatus,
  count(*) as Count
from employee
group by CurrentStatus
          """).show()

In [ ]:
# Вывести среднюю зп (OfferAmount) у Активных сотрудников в зависимости от Grade (округлить до 2 знаков после запятой)
spark.sql("""

          """).show()

In [ ]:
# 1) Посчитать средний возраст всех сотрудников.
spark.sql("""

          """).show()

In [ ]:
# 2) Посчитать количество мужчин и женщин в компании.
spark.sql("""

          """).show()

In [ ]:
# 3) Посчитать распределение сотрудников по отделам (Department).
spark.sql("""

          """).show()

In [ ]:
# 4*) Оценить уровень текучести кадров, посчитав процент уволенных сотрудников.
spark.sql("""

          """).show()

In [ ]:
# 5) Определить, как долго сотрудники работают в компании, используя разницу между текущей датой и датой их найма.
# from pyspark.sql.functions import current_date, datediff
spark.sql("""

          """).show()

In [ ]:
# 6) Определить средний доход специалистов в зависимости от уровня образования
spark.sql("""

          """).show()

In [ ]:
# 7) Посчитать среднюю продолжительность работы сотрудников в компании (с учетом даты найма).
spark.sql("""

          """).show()

In [ ]:
# 8) Преобразовать CurrentStatus в Int (Уволен = 0, Активен = 1)
spark.sql("""

          """).show()

# Новый DataFrame (Attendance)

In [ ]:
!ls -la | grep attendance

In [ ]:
# Считать DataFrame attendance.parquet


In [ ]:
# 9) Посчитать среднее количество отработанных часов для всех сотрудников.
spark.sql("""

          """).show()

In [ ]:
# 10) Посчитать долю сотрудников, которые в среднем работают больше 8 часов в день.
spark.sql("""

          """).show()

In [ ]:
df.groupBy("age").count().explain(True)

Оконные функции

In [ ]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("course").orderBy("income")
df1.withColumn("rn", F.row_number().over(windowSpec)).show()

Udf-функции

In [ ]:
@udf(returnType=StringType())
def upperCase(str):
    return str.upper()

df.withColumn("Cureated Name", upperCase(col("Name"))).show(truncate=False)

When-конструктор

In [ ]:
(covid_df
  .select('date', ‘location',
    when(col('total_cases') > 10000, 'Red zone')
    .when((col('total_cases') <= 10000) & (col('total_cases') > 5000), 'Yellow zone')
    .otherwise('Green zone').alias('zone'))
  .show())

In [ ]:
spark.stop()